In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
# import warnings filter
from warnings import simplefilter
#ignore all future warnings
simplefilter(action = 'ignore', category=FutureWarning)

from sklearn import preprocessing


In [2]:
churn= pd.read_csv('Churn.csv', na_values='?')
churn = churn.rename(index=str, columns={'Churn?':'Churn',"Int'l Plan":'Intl Plan'})
churn = churn.dropna()
churn.head()

,Unnamed: 0,State,Account Length,Area Code,Phone,Intl Plan,VMail Plan,VMail Message,Day Mins,Day Calls,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
3,3,OH,84,408.0,375-9999,yes,no,0,299.4,71,...,88.0,5.26,196.9,89,8.86,6.6,7.0,1.78,2,False.
4,4,OK,75,415.0,330-6626,yes,no,0,166.7,113,...,122.0,12.61,186.9,121,8.41,10.1,3.0,2.73,3,False.
5,5,AL,118,510.0,391-8027,yes,no,0,223.4,98,...,101.0,18.75,203.9,118,9.18,6.3,6.0,1.70,0,False.
6,6,MA,121,510.0,355-9993,no,yes,24,218.2,88,...,108.0,29.62,212.6,118,9.57,7.5,7.0,2.03,3,False.
9,9,WV,141,415.0,330-8173,yes,yes,37,258.6,84,...,111.0,18.87,326.4,97,14.69,11.2,5.0,3.02,0,False.


In [11]:
# converting the catagorical columns into integer values
label_encoder = preprocessing.LabelEncoder()
churn['State'] = label_encoder.fit_transform(churn['State'])
churn['Intl Plan'] = label_encoder.fit_transform(churn['Intl Plan'])
churn['VMail Plan'] = label_encoder.fit_transform(churn['VMail Plan'])
churn['Churn'] = label_encoder.fit_transform(churn['Churn'])

In [12]:
# one hot encoding of catagorical variables
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

catagories = ['State', 'Area Code']
catagorical_data = churn.loc[:,catagories]

enc.fit(catagorical_data)
one_hot = pd.DataFrame(enc.transform(catagorical_data).toarray())
one_hot.iloc[-3:]

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
1204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1205,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1206,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
# combining it with the data

churn_f = pd.concat([churn, one_hot], axis = 1)
churn_f.drop(["Phone","State","Area Code","Churn"], axis = 1, inplace = True)
churn_f.head()

,Unnamed: 0,Account Length,Intl Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,...,44,45,46,47,48,49,50,51,52,53
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [3]:
#model split for testing and training - std 75-25% split
X = churn.drop('Churn', axis=1)  
y = churn['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [4]:
#modeling

clf = LogisticRegression(C=1.0).fit(X_train, y_train)
#clf_predicted = clf.predict(X_test)
#confusion = confusion_matrix(y_test, clf_predicted)
#print('Logistic regression classifier\n', confusion)

ValueError: could not convert string to float: 'yes'

In [38]:
#cross validaton

scores = cross_val_score(clf, X, y, cv=10, scoring = 'roc_auc')
print('AUC:%0.2f' %(scores.mean()))

AUC:0.74


In [48]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
clf = SVC(kernel='rbf', gamma = 0.001).fit(X_train, y_train)
scores=cross_val_score(clf,X, y,cv=10,scoring = 'roc_auc')
print('AUC:%0.2f'%(scores.mean()))

AUC:0.71


In [59]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth = 5).fit(X_train, y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring = 'roc_auc')
print('AUC:%0.2f' %(scores.mean()))
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

AUC:0.80
Accuracy of Decision Tree classifier on training set: 0.93
Accuracy of Decision Tree classifier on test set: 0.92


In [107]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier().fit(X_train, y_train)
scores = cross_val_score(clf, X, y, cv=10, scoring = 'roc_auc')
print('AUC:%0.2f' %(scores.mean()))
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

AUC:0.85
Accuracy of Decision Tree classifier on training set: 0.95
Accuracy of Decision Tree classifier on test set: 0.92
